In [1]:
# imports

import torch
import pandas as pd
import pickle
import os
from tqdm import tqdm
from collections import OrderedDict

from dinov2_ood_utilities.imagenet_tree import ImagenetSemanticInfo, ImagenetSemanticSubtree
from dinov2_ood_utilities.custom_datasets import DictionaryDataset 



In [2]:
# define device to use

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device used: {device}')

Device used: cuda


In [3]:
# util objects

imagenet_info = ImagenetSemanticInfo()

Stop node discovered


In [11]:
with open('../resources/imagenet_1k_label_order.txt', 'r') as label_order_file:
    inet_1k_labels = label_order_file.readlines()
    inet_1k_labels = [label_order_line.split()[0] for label_order_line in inet_1k_labels]

In [12]:
# load model

pretr_head = torch.load('../resources/pretrained_heads/dinov2_vits14_linear_head.pth')

/tmp/ipykernel_2299079/2118242824.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretr_head = torch.load('../resources/pretrained_heads/dinov2_vits14_linear_head.pth')


In [9]:
from torch import nn

class LinearClassifier(nn.Module): 

    def __init__(self, in_features = 384, out_features = 1000):
        super().__init__()

        self.network = nn.Sequential(
            nn.Linear(in_features= in_features, out_features=out_features),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.network(x)
    

In [13]:
# create dataloaders for INet-1k-Val, INet-R, INet-V2

dataloaders_dict = dict()

for loader_name, loader_path in [('inet_1k_val', '../resources/vit_s_embeddings/inet_1k_val_cls_pt.pkl'),
                                 ('inet_r', '../resources/vit_s_embeddings/inet_r_cls_pt.pkl'),
                                 ('inet_v2_70', '../resources/vit_s_embeddings/inet_v2_70_cls_pt.pkl'),
                                 ('inet_v2_mf', '../resources/vit_s_embeddings/inet_v2_mf_cls_pt.pkl'),
                                 ('inet_v2_top', '../resources/vit_s_embeddings/inet_v2_top_cls_pt.pkl')]:
    
    with open(loader_path, 'rb') as pkl_fl:
        dataloaders_dict[loader_name] = torch.utils.data.DataLoader(dataset=DictionaryDataset(data=pickle.load(pkl_fl), index_list=inet_1k_labels), 
                                                   batch_size=128, shuffle=False, num_workers=8, pin_memory=True)



In [11]:
dataloaders_inet_c = dict()

inet_c_embeds_src_path = '../resources/vit_s_embeddings/imagenet_c'
inet_c_cor_types = os.listdir(inet_c_embeds_src_path)

for cor_type in inet_c_cor_types:
    for sev in range(1, 6):
        with open(f'{inet_c_embeds_src_path}/{cor_type}/sev_{sev}.pkl', 'rb') as pkl_fl: 
            if sev == 1:
                dataloaders_inet_c[cor_type] = dict()
            dataloaders_inet_c[cor_type][sev] = torch.utils.data.DataLoader(dataset=DictionaryDataset(pickle.load(pkl_fl), inet_1k_labels), 
                                                           batch_size=128, shuffle=False, num_workers=8, pin_memory=True) 

In [17]:
# helper functions that return only required data per batch item which depends on the used pretrained head

def cls_with_patch_one_layer(batch_samples: torch.Tensor) -> torch.Tensor:
    return torch.cat([torch.cat((batch_sample[1].squeeze(), batch_sample[0].squeeze())).unsqueeze(0)
                                            for batch_sample in batch_samples])

def cls_with_patch_one_layer_inet_c(batch_samples: torch.Tensor) -> torch.Tensor: 
    return torch.cat([torch.cat((batch_sample[1].squeeze(), batch_sample[0].squeeze())).unsqueeze(0) for batch_sample in batch_samples])

def cls_without_patch_one_layer(batch_samples: torch.Tensor) -> torch.Tensor: 
    return torch.cat([batch_sample[1] for batch_sample in batch_samples])

def cls_without_patch_one_layer_inet_c(batch_samples: torch.Tensor) -> torch.Tensor: 
    return torch.cat([batch_sample[1] for batch_sample in batch_samples])

def cls_with_patch_four_layers(batch_samples: torch.Tensor) -> torch.Tensor: 
    return torch.cat([torch.cat(((torch.cat([cls_token for _, cls_token in batch_sample], dim=-1),
                    batch_sample[3][0])), dim=-1)
                    for batch_sample in batch_samples])

# helper function 
# takes: model and list of dataloaders
# returns: accuracy for each dataloader

def calc_accuracy(model: torch.nn.Sequential, n_layers: int, with_patch: bool, dataloaders: list, is_inet_c_data: bool) -> list: 

    if n_layers == 1:
        if with_patch:
            sample_transform = cls_with_patch_one_layer 
            if is_inet_c_data:
                sample_transform = cls_with_patch_one_layer_inet_c 
        else:
            sample_transform = cls_without_patch_one_layer 
            if is_inet_c_data:
                sample_transform = cls_without_patch_one_layer_inet_c
    elif n_layers == 4: 
        sample_transform = cls_with_patch_four_layers 

    if sample_transform == None:
        raise ValueError('SampleTransform function could not be defined')
    
    accuracies = []
    
    for dataloader in (pbar := tqdm(dataloaders, ncols=100)):
                  
        total_preds = 0
        total_preds_true = 0
        for batch_samples, batch_labels in dataloader: 
            batch_samples_dev = sample_transform(batch_samples).to(device)
            
            model_pred = model(batch_samples_dev).cpu()

            is_equal = model_pred.argmax(axis=1) == batch_labels.argmax(axis=1)
            total_preds += is_equal.shape[0]
            total_preds_true += is_equal.type(torch.float).sum().item()

        accuracies.append(total_preds_true / total_preds)

    return accuracies 
            

In [15]:
# definition of linear classifiers 

model_params_one_layer = OrderedDict()
model_params_four_layers = OrderedDict()
model_params_cls_token = OrderedDict()
model_params_one_layer['network.0.weight'] = pretr_head['weight']
model_params_one_layer['network.0.bias'] = pretr_head['bias']
model_params_cls_token['network.0.weight'] = pretr_head['weight'][:,0:384]
model_params_cls_token['network.0.bias'] = pretr_head['bias']

lc_one_layer = LinearClassifier(in_features=768, out_features=1000)
lc_cls_token = LinearClassifier(in_features=384, out_features=1000)

lc_one_layer.load_state_dict(model_params_one_layer)
lc_cls_token.load_state_dict(model_params_cls_token)

lc_one_layer.eval()
lc_cls_token.eval()

lc_one_layer.to(device)
lc_cls_token.to(device)

LinearClassifier(
  (network): Sequential(
    (0): Linear(in_features=384, out_features=1000, bias=True)
    (1): Softmax(dim=1)
  )
)

In [17]:
# Calculate acc for each Imagenet-C Val dataset 

for model, model_name, store_path in [(lc_cls_token, 'lc_cls', '../results/lc_cls_pretr_inet_c_res.pkl'),
                          (lc_one_layer, 'lc_cls_patch', '../results/lc_one_lay_pretr_inet_c_res.pkl')]:
    
    inet_c_accuracies = dict()
    inet_c_dloader_list = []

    # list of dataloaders
    for cor_type in dataloaders_inet_c.keys():
        for sev in dataloaders_inet_c[cor_type].keys():
            inet_c_dloader_list.append(dataloaders_inet_c[cor_type][sev])

    # get list acc-vals, one acc-value for each of 95 INet-C datasets
    if model_name == 'lc_cls':
        inet_c_acc_list = calc_accuracy(model, 1, False, inet_c_dloader_list, is_inet_c_data=True)
    else: 
        inet_c_acc_list = calc_accuracy(model, 1, True, inet_c_dloader_list, is_inet_c_data=True)

    # map each acc-value to combination of (corruption type, severity)
    acc_res_i = 0
    for cor_type in dataloaders_inet_c.keys():
        for sev in dataloaders_inet_c[cor_type].keys():
            if sev == 1:
                inet_c_accuracies[cor_type] = dict()
            inet_c_accuracies[cor_type][sev] = inet_c_acc_list[acc_res_i]
            acc_res_i += 1 

    with open(store_path, 'wb') as pkl_fl:
        pickle.dump(inet_c_accuracies, pkl_fl, pickle.HIGHEST_PROTOCOL)

100%|███████████████████████████████████████████████████████████████| 95/95 [07:38<00:00,  4.83s/it]


In [8]:
# one DF with INet-C accuracies  

inet_c_acc_dfs = dict()

for model, load_path in [('lc_with_patch', '../results/lc_one_lay_pretr_inet_c_res.pkl'),
              ('lc_cls', '../results/lc_cls_pretr_inet_c_res.pkl')]: 
    # load dict with acc of 95 Inet-C Datasets 
    with open(load_path, 'rb') as pkl_fl:
        inet_c_acc_results = pickle.load(pkl_fl) 

    # create tuples with form: (dataset, cor, sev, model, acc)
    inet_c_df_data = []
    for cor_type in inet_c_acc_results.keys(): 
        for sev in inet_c_acc_results[cor_type].keys(): 
            acc = inet_c_acc_results[cor_type][sev]
            inet_c_df_data.append(('inet-c', cor_type, sev, model, acc))

    # create DF 
    inet_c_acc_dfs[model] = pd.DataFrame(data=inet_c_df_data, columns=['dataset', 'cor_type', 'sev', 'model', 'acc'])
    print(inet_c_acc_dfs[model].head())

  dataset cor_type  sev          model      acc
0  inet-c  spatter    1  lc_with_patch  0.78862
1  inet-c  spatter    2  lc_with_patch  0.75822
2  inet-c  spatter    3  lc_with_patch  0.71684
3  inet-c  spatter    4  lc_with_patch  0.72546
4  inet-c  spatter    5  lc_with_patch  0.67486
  dataset cor_type  sev   model      acc
0  inet-c  spatter    1  lc_cls  0.72488
1  inet-c  spatter    2  lc_cls  0.69854
2  inet-c  spatter    3  lc_cls  0.66492
3  inet-c  spatter    4  lc_cls  0.66916
4  inet-c  spatter    5  lc_cls  0.63050


In [5]:
# bring inet-c-acc-DF in form (datasetname + sev, model-1-acc, model-2-acc)

inet_c_avg_acc = inet_c_acc_dfs['lc_with_patch'].groupby(['sev'])[['acc']].mean()
inet_c_avg_acc.rename({'acc': 'patch-acc'}, axis='columns', inplace=True)
acc_cls = inet_c_acc_dfs['lc_cls'].groupby(['sev'])[['acc']].mean()
inet_c_avg_acc['cls-acc'] = acc_cls['acc']
inet_c_avg_acc = inet_c_avg_acc.assign(dataset= lambda x: x.index)
inet_c_avg_acc['dataset'] = inet_c_avg_acc['dataset'].apply(lambda x: 'inet-c-sev-' + str(x))
print(inet_c_avg_acc)

     patch-acc   cls-acc       dataset
sev                                   
1     0.739889  0.677464  inet-c-sev-1
2     0.673617  0.615355  inet-c-sev-2
3     0.599676  0.548494  inet-c-sev-3
4     0.497449  0.456148  inet-c-sev-4
5     0.367285  0.338309  inet-c-sev-5


In [18]:
# calculate results for inet-r, inet-v2-70, inet-v2-mf, inet-v2-top 

ood_acc = dict() 

for model, model_name in [(lc_one_layer, 'lc_with_patch'), (lc_cls_token, 'lc_cls')]: 
    ood_acc[model_name] = dict()
    for dname in dataloaders_dict.keys(): 
        if model_name == 'lc_with_patch':
            ood_acc[model_name][dname] = calc_accuracy(model, 1, True, [dataloaders_dict[dname]], is_inet_c_data=False)[0]
        else: 
            ood_acc[model_name][dname] = calc_accuracy(model, 1, False, [dataloaders_dict[dname]], is_inet_c_data=False)[0]


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.78it/s]


In [19]:
# create DF for ood-acc-dict: (dset, model, acc)

ood_acc_data = []
for model_name in ood_acc.keys():
    for dset_name in ood_acc[model_name].keys():
        ood_acc_data.append((dset_name, model_name, ood_acc[model_name][dset_name]))

ood_acc_df = pd.DataFrame(data=ood_acc_data, columns=['dataset', 'model', 'acc'])
acc_with_patch = ood_acc_df.loc[ood_acc_df['model']=='lc_with_patch']
acc_cls = ood_acc_df.loc[ood_acc_df['model']=='lc_cls']
acc_cls = acc_cls.set_index([pd.Index([0, 1, 2, 3, 4])])
acc_ood_df = acc_with_patch[['dataset', 'acc']]
acc_ood_df.rename(mapper={'acc': 'patch-acc'}, axis='columns', inplace=True)
acc_ood_df['cls-acc'] = acc_cls['acc']

# concatenate inet-c results 
acc_ood_all = pd.concat([inet_c_avg_acc, acc_ood_df])
acc_ood_all.to_csv('../results/ood_acc_results.csv', sep=';')

/tmp/ipykernel_2299079/2315577759.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_ood_df.rename(mapper={'acc': 'patch-acc'}, axis='columns', inplace=True)


In [2]:
# combine avg measure values with accuracy values of linear classifier 

map_dname = {'inet-c-sev-1': 'inet_c_sev_1', 'inet-c-sev-2': 'inet_c_sev_2', 'inet-c-sev-3': 'inet_c_sev_3', 'inet-c-sev-4': 'inet_c_sev_4',
             'inet-c-sev-5': 'inet_c_sev_5', 'inet_r': 'inet_r', 'inet_v2_70': 'inet_v2_70', 'inet_v2_mf': 'inet_v2_mf', ''
             'inet_v2_top': 'inet_v2_top'}

ood_acc_df = pd.read_csv('../results/ood_acc_results.csv', sep=';')
avg_measure_all = pd.read_csv('../results/class_statistics_avg_results.csv', sep=';')

avg_measure_all.set_index(avg_measure_all['dataset'], inplace=True)

ood_acc_df = ood_acc_df.loc[ood_acc_df['dataset'] != 'inet_1k_val']
ood_acc_df['dataset'] = ood_acc_df.apply(lambda x: map_dname[x['dataset']], axis=1)
ood_acc_df.set_index(ood_acc_df['dataset'], inplace=True)
ood_acc_df.drop('Unnamed: 0', axis='columns', inplace=True)

ood_acc_and_measures_df = pd.concat([ood_acc_df, avg_measure_all], axis=1)
ood_acc_and_measures_df.drop('dataset', axis=1, inplace=True)

ood_acc_and_measures_df['CLS'] = ood_acc_and_measures_df['cls-acc']
ood_acc_and_measures_df['Patch + CLS'] = ood_acc_and_measures_df['patch-acc']
ood_acc_and_measures_df.drop('patch-acc', axis=1, inplace=True)
ood_acc_and_measures_df.drop('cls-acc', axis=1, inplace=True)
ood_acc_and_measures_df.rename({ 'c2': 'C2 measure', 'c3': 'C3 Measure', 'c1': 'C1 Measure', 'l1_measure': 'L1 Measure'},
                               inplace=True, axis=1)

col_old = ood_acc_and_measures_df.columns
ood_acc_and_measures_df.set_index(pd.Index(['ImageNet-C-Sev-1', 'ImageNet-C-Sev-2', 'ImageNet-C-Sev-3', 'ImageNet-C-Sev-4', 'ImageNet-C-Sev-5',
                                            'ImageNet-R', 'ImageNet-V2-70', 'ImageNet-V2-MF', 'ImageNet-V2-TOP']), inplace=True)
ood_acc_and_measures_df.columns = pd.MultiIndex.from_tuples([
    ('Measure', col_old[0]),
    ('Measure', col_old[1]),
    ('Measure', col_old[2]),
    ('Measure', col_old[3]),
    ('Accuracy', col_old[4]), 
    ('Accuracy', col_old[5])])

# create table with background colour of cell indicating the value rank
with open('/home/stud/afroehli/coding/dinov2_ood/diagrams/ood_acc_and_measures.tex', 'w') as latex_table:
    ood_acc_and_measures_df.style.background_gradient(cmap='viridis').background_gradient(cmap='viridis_r', subset=[('Measure', 'L1 Measure')]).set_table_styles([
        {"selector": "th, td", "props": [("font-size", "20px"), ("padding", "12px")]},
        {'selector': 'th.col_heading.level0', 'props': [('text-align', 'center')]},
        {'selector': 'th.col_heading.level1', 'props': [('text-align', 'center')]},
        {
            'selector': 'th.col_heading.level1:2',
            'props': [('border-right', '4px solid black')]
        },
        {
            'selector': 'td:3',
            'props': [('border-right', '4px solid black')]
        }
    ]).to_latex(latex_table)

ood_acc_and_measures_df.style.background_gradient(cmap='viridis').background_gradient(cmap='viridis_r', subset=[('Measure', 'L1 Measure')]).set_table_styles([
        {"selector": "th, td", "props": [("font-size", "20px"), ("padding", "12px")]},
        {'selector': 'th.col_heading.level0', 'props': [('text-align', 'center')]},
        {'selector': 'th.col_heading.level1', 'props': [('text-align', 'center')]},
        {
            'selector': 'th.col_heading.level1:2',
            'props': [('border-right', '4px solid black')]
        },
        {
            'selector': 'td:3',
            'props': [('border-right', '4px solid black')]
        }
    ])